# Введение

Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:

Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
Контролировать добросовестность кассиров при продаже алкоголя.

Постройте модель, которая по фотографии определит приблизительный возраст человека. В вашем распоряжении набор фотографий людей с указанием возраста.

Данная задача является задачей регрессии, целевой метрикой является метрика MAE, ее значение должно составлять меньше 8.

# Исследовательский анализ данных

In [4]:
# импорт билиотек
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')

In [ ]:
# размер датасета
df.shape

In [ ]:
df.info()

In [ ]:
# график распределения людей по возрасту
df['real_age'].hist()
plt.title("Распрделение возраста")
plt.xlabel("Возраст")
plt.ylabel("Количество людей")

In [2]:
# извлечение картинок
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
features, target = next(train_gen_flow)

# выводим 15 изображений
fig = plt.figure(figsize=(10,10))
for i in range(15):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

**Вывод**

Был проведен исследовательский анализ данных, в ходе которого был сделан вывод о том, что больше всего людей в распределении относится к возрасту 20-30 лет. В датасете отсутствуют пропуски. Также следует отметить, что в датасете присутствуют как цветные, так и черно-белые фотографии, некоторые из них повернуты.

Для улучшения обучения модели необходимо уменьшить влияние этих факторов: добавляется аугментация, увеличивается количество эпох и снижается скорость обучения модели.

# Обучение модели

In [ ]:
# функция загрузки тренировочных данных

def load_train(path):
    labels = pd.read_csv('/datasets/faces/labels.csv')

    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        validation_split=0.25,
        horizontal_flip=True,
        vertical_flip=True)

    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)

    return train_gen_flow

In [ ]:
# функция загрузки тестовых данных

def load_test(path):

    labels = pd.read_csv('/datasets/faces/labels.csv')

    test_datagen = ImageDataGenerator(
        rescale=1.0/255,
        validation_split=0.25)

    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)

    return test_gen_flow

In [ ]:
# функция создания модели

def create_model(input_shape):

    backbone = ResNet50(input_shape=input_shape,
        weights='imagenet',
        include_top=False)

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(1, activation='relu'))

    optimizer = Adam(lr=0.0001)
    model.compile(loss='mse',optimizer=optimizer, metrics=['mae'])

    return model

In [ ]:
# функция обучения модели

def train_model(model, train_data, test_data, batch_size=None, epochs=10,
    steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
        validation_data=test_data,
        batch_size=batch_size, epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2)

    return model

'''Train for 356 steps, validate for 119 steps
Epoch 1/10
2023-07-28 14:51:07.206225: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-07-28 14:51:07.489680: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7

356/356 - 54s - loss: 240.7817 - mae: 11.4654 - val_loss: 598.7144 - val_mae: 19.2781
Epoch 2/10
356/356 - 71s - loss: 114.4266 - mae: 8.0991 - val_loss: 162.8076 - val_mae: 9.5016

Epoch 3/10
356/356 - 67s - loss: 89.8226 - mae: 7.1959 - val_loss: 197.0861 - val_mae: 11.1104

Epoch 4/10
356/356 - 68s - loss: 73.8645 - mae: 6.5884 - val_loss: 83.8716 - val_mae: 6.9288

Epoch 5/10
356/356 - 79s - loss: 60.9170 - mae: 5.9545 - val_loss: 84.4396 - val_mae: 6.9547

Epoch 6/10
356/356 - 84s - loss: 54.4624 - mae: 5.6239 - val_loss: 79.2450 - val_mae: 6.6180

Epoch 7/10
356/356 - 74s - loss: 46.0303 - mae: 5.1974 - val_loss: 91.2569 - val_mae: 7.0422

Epoch 8/10
356/356 - 82s - loss: 41.0021 - mae: 4.8876 - val_loss: 83.8227 - val_mae: 6.7788

Epoch 9/10
356/356 - 59s - loss: 35.6409 - mae: 4.5710 - val_loss: 94.7778 - val_mae: 7.3930

Epoch 10/10
356/356 - 48s - loss: 29.9138 - mae: 4.1993 - val_loss: 71.2509 - val_mae: 6.2757

WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
  
119/119 - 11s - loss: 71.2509 - mae: 6.2757
Test MAE: 6.2757'''

**Вывод**

Итоговый MAE составил 6,27, что является удовлетворительным результатом

# Общий вывод

В ходе выполнения данного проекта была обучена модель компьютерного зрения, которая определяет возраст людей. Итоговый MAE обученной модели составил 6,27, что является удовлетворительным результатом.

Также следует отметить, что в датасете были выделены следющие проблемы: цветные и черно-белые фотографии, а также различная ориентация фотографий (некоторые повернуты).

Для улучшения обучения модели были предприняты следующие методы: использовалась модель компьютерного зрения ResNet50, была проведена аугментация, количество эпох обучения увеличено до 10, скорость обучения модели составляет 0,0001. Также фотографии были приведены к единому масштабу от 0 до 1.

<div class="alert alert-info"> <b>Комментарий студента:</b> Здравствуйте, хотел бы у Вас попросить какие-либо материалы, либо совет касательно компьютерного зрения.

На работе поставили задачу создать программу, которая из отсканированных файлов PDF могла бы выделять таблицы, и эти таблицы заносить в Excel, то есть необходимо делать оцифровку изображений.

Хотел бы у Вас попросить какую-то литература, или, может быть, совет по этой задаче.

Премного буду благодарен. Заранее спасибо.</div>